In [ ]:
import pandas as pd

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f transport-all-comments.csv.xz || wget  https://github.com/heiseacademy/ml-python/raw/main/06-ml-projekt/transport-all-comments.csv.xz")


In [ ]:
posts = pd.read_csv("transport-all-comments.csv.xz", parse_dates=["created_utc"])

In [ ]:
from collections import Counter

In [ ]:
!pip install regex tqdm

In [ ]:
import regex as re
from tqdm.auto import tqdm
title_counter = Counter([w.lower() for t in tqdm(posts["text"]) for w in re.findall(r'[\w-]*\p{L}[\w-]*', t)])

In [ ]:
!pip install wordcloud

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
wc.generate_from_frequencies(title_counter)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')

In [ ]:
!pip install spacy

In [ ]:
import spacy

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
for w in "removed deleted post message account moderators http https www youtube com \
          watch gt look looks feel test know think go going submission link apologize \
          inconvenience don want automatically based buy compose good image karma like \
          lot need people self shit sound sounds spam submitting subreddit things \
          video way years time days doesn en fuck money org read reddit review \
          right said says subreddit subreddits sure thank try use videos wiki \
          wikipedia work ll thing point ve actually wait hello new amp better \
          isn yeah probably pretty yes didn pay long posts commenting portion \
          contribute questions unfortunately allowed submissions gifs pics sidebar".split(" "):
    stopwords.add(w)

In [ ]:
title_counter = Counter([w for t in tqdm(posts["text"].str.lower())
                            for w in re.findall(r'[\w-]*\p{L}[\w-]*', t)
                               if (w not in stopwords) and (len(w) > 1)
                        ])

In [ ]:
!pip install wordcloud

In [ ]:
wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
wc.generate_from_frequencies(title_counter)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')

In [ ]:
text_counter = Counter([w for t in tqdm(posts[posts["parent_id"].isna()]["text"].str.lower()) 
                            for w in re.findall(r'[\w-]*\p{L}[\w-]*', t)
                               if (w not in stopwords) and (len(w) > 1)
                        ])

In [ ]:
wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
wc.generate_from_frequencies(text_counter)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_text_vectorizer = TfidfVectorizer(stop_words=list(stopwords), min_df=5, max_df=0.7)
tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(posts['text'])

In [ ]:
from sklearn.decomposition import NMF
nmf_text_model = NMF(n_components=10, random_state=42)
W_text_matrix = nmf_text_model.fit_transform(tfidf_text_vectors)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def wordcloud_topic_model_summary(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        freq = {}
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            freq[feature_names[i].replace(" ", "_")] = topic[i]
        wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
        wc.generate_from_frequencies(freq)
        plt.figure(figsize=(12,12))
        plt.imshow(wc, interpolation='bilinear')

In [ ]:
wordcloud_topic_model_summary(nmf_text_model, tfidf_text_vectorizer.get_feature_names_out(), 40)